In [1]:
import requests
import json
import urllib
import  urllib.parse
import pandas as pd
import getpass
from requests.auth import HTTPBasicAuth
token = ''

In [2]:
from ikewaipy.ikewai import Ikewai

In [3]:
#Setup a client object for connecting to Ike Wai so we can get auth token
ike = Ikewai(username='ckkondo')

In [4]:
#login with your UH credentials
ike.login()

 ············


Login  Successful - token has been set and will be valid for 240 minutes. You can now access Ike Wai data.


In [5]:
#this is how we access our Auth Token
ike.token

'e2c8566a2da2c814712466b55edd4dc6'

In [6]:
#inject endpoint
endpoint = 'ikeauth.its.hawaii.edu'

In [43]:
#This is how we use the ikewaipy search for JSON metadata objects - similiar to listMetadata

ike.searchMetadata("{'name':'Micro-GPS','value.Location':'Lepelau'}", 10)

#ike.searchMetadata("{'name':'Micro_GPS'}", 1)

[{'uuid': '2436755676838171116-242ac1110-0001-012',
  'owner': 'seanbc',
  'schemaId': None,
  'internalUsername': None,
  'associationIds': [],
  'lastUpdated': '2022-03-22T22:41:32.774-05:00',
  'name': 'Micro-GPS',
  'value': {'x': 231,
   'Location': 'Lepelau',
   'Watershed': 'KOHOLALELE',
   'Site_Enviro': 'Freshwater',
   'Site_Type': 'Stream',
   'Site_Salinity': 'Freshwater',
   'Lattitude': 20.0212343,
   'Longitude': -155.3598134,
   'Elevation': 417.6,
   'Max_Elevation_Watershed-m': 1964.0,
   'WSP': 0.212627291,
   'GROUP_bywatershedpercent': 'MID.1',
   'GROUP_byelevation': 'MAUKA.3',
   'Distribution': 1,
   'Watershed_Order': 'KOHOLALELE_1',
   'Mauka': 'MAUKA',
   'norm_order': 3,
   'norm_watershed_order': 'KOHOLALELE_3',
   'watershed_mauka': 'KOHOLALELE_MAUKA',
   'type_norm_order': '3.Urban_KOHOLALELE_3',
   'loc': {'type': 'Point', 'coordinates': [-155.3598134, 20.0212343]}},
  'created': '2022-03-22T22:41:32.774-05:00',
  '_links': {'self': {'href': 'https://ike

In [44]:
ike.searchMetadata("{'name':'GPS'}", 1)

[{'uuid': '3601472873912921620-242ac1110-0001-012',
  'owner': 'ckkondo',
  'schemaId': None,
  'internalUsername': None,
  'associationIds': [],
  'lastUpdated': '2022-03-22T18:28:39.800-05:00',
  'name': 'GPS',
  'value': {'x': 18,
   'location': 'Honokohau.Right.bowl',
   'watershed': 'HONOKOHAU',
   'site_enviro': 'Freshwater',
   'site_type': 'Waterfall',
   'site_salinity': 'Freshwater',
   'lattitude': 20.90891,
   'longitude': -156.58375,
   'elevation': 778.5,
   'max_elevation_watershed-m': 1761.0,
   'wsp': 0.442078365,
   'group_by_water_shed_percent': 'MAUKA.3',
   'group_by_elevation': 'MAUKA.2',
   'distribution': 1,
   'watershed_order': 'HONOKOHAU_1',
   'mauka': 'MAUKA',
   'norm_order': 3,
   'norm_watershed_order': 'HONOKOHAU_3',
   'watershed_mauka': 'HONOKOHAU_MAUKA',
   'type_norm_order': '1.Pristine_HONOKOHAU_3',
   'loc': {'type': 'Point', 'coordinates': [-156.58375, 20.90891]}},
  'created': '2022-03-22T18:28:39.800-05:00',
  '_links': {'self': {'href': 'https

In [ ]:
#This is how we use the ikewaipy search for JSON metadata objects - similiar to listMetadata

#ike.searchMetadata("{'name':'Variable','value.variable_name':'Temperature'}", 1)

ike.searchMetadata("{'name':'Well'}", 1)

In [ ]:
#HTTP REST call to meta API with a mongo style query
def listMetadata(token, query="", limit=10, offset=0):
    safe_query = urllib.parse.quote(query.encode('utf8'))
    headers = {
        'authorization': "Bearer " + token,
        'content-type': "application/json",
    }
    res = requests.get('https://'+endpoint+'/meta/v2/data?q='+safe_query+'&limit='+str(limit)+'&offset='+str(offset), headers=headers,verify=False)
    resp = json.loads(res.content)
    return resp['result']

In [ ]:
#HTTP API call to meta to fetch a single metadata JSON document by it's uuid
def getMetadata(token, uuid):
    headers = {
        'authorization': "Bearer " + token,
        'content-type': "application/json",
    }
    res = requests.get('https://'+endpoint+'/meta/v2/data/'+uuid, headers=headers,verify=False)
    resp = json.loads(res.content)
    return resp['result']

In [ ]:
#HTTPS API call to update an existing metadata JSON document - note you have to pass every field as this is not a PATCH
def updateMetadata(token, uuid, data):
    headers = {
        'authorization': "Bearer " + token,
        'content-type': "application/json",
    }
    res = requests.post('https://'+endpoint+'/meta/v2/data/'+uuid, json=data, headers=headers,verify=False)
    resp = json.loads(res.content)
    return resp

In [ ]:
#CREATE a new metadata document
def createMetadata(token, data):
    headers = {
        'authorization': "Bearer " + token,
        'content-type': "application/json",
        
    }
    res = requests.post('https://'+endpoint+'/meta/v2/data/', json=json.loads(data), headers=headers,verify=False)
    print(res.content)
    resp = json.loads(res.content)
    return resp

In [ ]:
#permission needs to be READ READ-WRITE or ALL
def updateMetadataPem(token, uuid, username, permission):
    headers = {
        'authorization': "Bearer " + token,
        'content-type': "application/json",
        
    }
    data = {'permission':permission}
    res = requests.post('https://'+endpoint+'/meta/v2/data/'+uuid+'/pems/'+username, json=data, headers=headers,verify=False)
    resp = json.loads(res.content)
    return resp
    

In [ ]:
def getMetadataPems(token,uuid):
    headers = {
        'authorization': "Bearer " + token,
        'content-type': "application/json",
    }
    res = requests.get('https://'+endpoint+'/meta/v2/data/'+uuid+'/pems/', headers=headers,verify=False)
    resp = json.loads(res.content)
    return resp

In [21]:
#load csv in a dataframe example and see dataframe
df1 = pd.read_csv('GPS.csv')
df1


,x,location,watershed,site_enviro,site_type,site_salinity,lattitude,longitude,elevation,max_elevation_watershed-m,wsp,group_by_water_shed_percent,group_by_elevation,distribution,watershed_order,mauka,norm_order,norm_watershed_order,watershed_mauka,type_norm_order
0,1,Honolii,HONOLII,Marine,Ocean,Marine,19.756377,-155.090424,0.0,39.2,0.000000,KAI,KAI,1,HONOLII_1,KAI,10,HONOLII_10,HONOLII_KAI,3.Urban_HONOLII_10
1,2,Onekahakaha,KEAUKAHA,Marine,Ocean,Marine,19.738583,-155.039120,0.0,6.6,0.000000,KAI,KAI,1,KEAUKAHA_1,KAI,10,KEAUKAHA_10,KEAUKAHA_KAI,3.Urban_KEAUKAHA_10
2,3,Puhi_Bay,KEAUKAHA,Marine,Ocean,Marine,19.732241,-155.046373,0.0,6.6,0.000000,KAI,KAI,1,KEAUKAHA_1,KAI,10,KEAUKAHA_10,KEAUKAHA_KAI,3.Urban_KEAUKAHA_10
3,4,Richardson,KEAUKAHA,Marine,Ocean,Marine,19.736927,-155.013564,0.0,6.6,0.000000,KAI,KAI,1,KEAUKAHA_1,KAI,10,KEAUKAHA_10,KEAUKAHA_KAI,3.Urban_KEAUKAHA_10
4,5,Kalapahapuu,KOHOLALELE,Freshwater,Stream,Freshwater,20.033667,-155.355350,219.4,1964.0,0.111711,MID.2,MID.1,2,KOHOLALELE_2,KALA,5,KOHOLALELE_5,KOHOLALELE_KALA,3.Urban_KOHOLALELE_5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,227,N3_KahaluuB,KEAUHOU,Freshwater,Groundwater,Freshwater,19.581667,-155.949722,254.9,1571.0,0.162253,MID.2,MAUKA.3,8,KEAUHOU_8,KALA,4,KEAUHOU_4,KEAUHOU_KALA,2.Aggriculture_KEAUHOU_4
227,228,N3_KahaluuA,KEAUHOU,Freshwater,Groundwater,Freshwater,19.583056,-155.949722,253.5,1571.0,0.161362,MID.2,MAUKA.3,8,KEAUHOU_8,KALA,4,KEAUHOU_4,KEAUHOU_KALA,2.Aggriculture_KEAUHOU_4
228,229,N3_KahaluuShaft,KEAUHOU,Freshwater,Groundwater,Freshwater,19.581356,-155.954122,178.3,1571.0,0.113495,MID.2,MID.1,9,KEAUHOU_9,KALA,4,KEAUHOU_4,KEAUHOU_KALA,2.Aggriculture_KEAUHOU_4
229,230,Koholalele,KOHOLALELE,Marine,Ocean,Marine,20.048624,-155.350672,3.9,1964.0,0.001986,MAKAI,MAKAI,3,KOHOLALELE_3,MAKAI,7,KOHOLALELE_7,KOHOLALELE_MAKAI,3.Urban_KOHOLALELE_7


In [ ]:
#Parse a CSV file and convert it to JSON metadata documents

#read in csv file to a pandas dataframe
df1 = pd.read_csv('GPS.csv')

#set static json body values and permsissions
#users needs to exist in agave
body={}
pem1={}
pem1['username']= 'chazk'
pem1['permission']='ALL'
pem2={}
pem2['username']= 'jgeis'
pem2['permission']='ALL'
pem4={}
pem4['username']= 'ikewai-admin'
pem4['permission']='ALL'
pem5={}
pem5['username']= 'public'
pem5['permission']='READ'

#the name field should be the larger document type we want - so this should be akin to the CSV file - we will use this for searching for these document types
body['name'] = "GPS"

#add our permissions to the document body so they are set at creation
body['permissions']=[pem1,pem2,pem4,pem5]


#should loop through each dataframe row convert to json 
i=0
ls = [0,1,2]
for i in df1.index:
    j = df1.loc[i].to_json()
    js = json.loads(j)
    #This stores a GeoJSON object in the value.loc field - in Ike Wai this has a spatial index on it in mongodb
    #ONLY DO THIS FOR georeferenced documents - so probably just one CSV
    js['loc'] = {"type":"Point", "coordinates":[js['longitude'],js['lattitude']]}
    body['value'] = js
    body['geospatial']= True;
    #preview our JSON document
    # print(body)
    #create the document in Ike Wai - 
    createMetadata(ike.token,json.dumps(body))
    

In [15]:
#read in csv file to a pandas dataframe
siteDateDf = pd.read_csv('./csv/Site_Date.csv')
geochemDf = pd.read_csv('./csv/Geochem.csv')

# applying merge
merged = pd.merge(siteDateDf, geochemDf, on = ['id'], how = 'outer')

In [16]:
merged.to_csv('Site_Date_Geochem.csv', encoding='utf-8')